In [1]:
import torch
from torchtext import data
import torch.nn as nn
import time
import math

In [2]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', init_token = '<sos>', eos_token = '<eos>', lower = True)
LABEL = data.LabelField(dtype = torch.float)

In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [7]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [23]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)

# TEXT.build_vocab(train_data, 
#                  max_size = MAX_VOCAB_SIZE, 
#                  vectors = "glove.6B.100d", 
#                  unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [91]:
print(vars(train_data.examples[0])['text'][:32])

['once', 'big', 'action', 'star', 'who', 'fell', 'off', 'the', 'face', 'of', 'the', 'earth', 'ends', 'up', 'in', 'a', 'small', 'town', 'with', 'a', 'problem', 'with', 'drug', 'dealers', 'and', 'a', 'dead', 'body', 'of', 'a', 'federal', 'agent']


In [12]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25004
Unique tokens in LABEL vocabulary: 2


In [14]:
# print(TEXT.vocab.freqs.most_common(20))
print(TEXT.vocab.itos[:10])
# print(LABEL.vocab.stoi)

['<unk>', '<pad>', '<sos>', '<eos>', 'the', ',', '.', 'and', 'a', 'of']


In [15]:
class RNNpart1(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
    
    def forward(self, text):
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        return hidden.squeeze(0) # [batch_size, hid dim]

In [16]:
class RNNpart2(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super().__init__()
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, hidden_squeeze):
        return self.fc(hidden_squeeze)

In [17]:
BATCH_SIZE = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [59]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

encoder = RNNpart1(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, PAD_IDX)
classifer = RNNpart2(HIDDEN_DIM, OUTPUT_DIM)
# print(encoder)
# print(classifer)

In [21]:
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad) 

# print(f'The model has {count_parameters(encoder):,} trainable parameters')

The model has 2,592,048 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)
encoder.embedding.weight.data.copy_(pretrained_embeddings)

In [42]:
import torch.optim as optim
import itertools
# params = [model1.parameters(), model2.parameters()]
params = itertools.chain(encoder.parameters(), classifer.parameters())

optimizer = optim.SGD(params, lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

encoder = encoder.to(device)
classifer = classifer.to(device)

criterion = criterion.to(device)

In [43]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [44]:
def train(encoder, classifer, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    encoder.train()
    classifer.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        hidden_vec = encoder(batch.text) 
        predictions = classifer(hidden_vec).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [45]:
def evaluate(encoder, classifer, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model1.eval()
    model2.eval()

    with torch.no_grad():
        for batch in iterator:
#             predictions = model2(model1(batch.text)).squeeze(1)
            hidden_vec = encoder(batch.text) 
            predictions = classifer(hidden_vec).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [35]:
# encoder.embedding.weight.requires_grad = False

In [46]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [47]:
N_EPOCHS = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(encoder, classifer, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(encoder, classifer, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder.state_dict(), 'encoder.pt')
        torch.save(classifer.state_dict(), 'classifer.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 9m 9s
	Train Loss: 0.693 | Train Acc: 50.52%
	 Val. Loss: 0.694 |  Val. Acc: 50.73%


In [48]:
# encoder = RNNpart1(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, PAD_IDX)
# classifer = RNNpart2(HIDDEN_DIM, OUTPUT_DIM)
encoder.load_state_dict(torch.load('encoder.pt'))
classifer.load_state_dict(torch.load('classifer.pt'))

encoder.to(device)
classifer.to(device)

test_loss, test_acc = evaluate(model1, model2, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.687 | Test Acc: 56.23%


In [78]:
class RNNdecoder(nn.Module):
    
    def __init__(self, hidden_dim, embedding_dim, vocab_size, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, hidden, text):
        #text = [batch size]
        hidden = hidden.unsqueeze(0)
        #hidden = [n layers * n directions = 1, batch size, hid dim]
        inp = text.unsqueeze(0)
        #inp = [1, batch size]
        embedded = self.embedding(inp)
        #embedded = [1, batch size, emb dim]
        output, hidden = self.rnn(embedded,hidden)
        #output = [seq len=1, batch size, hid dim * (n directions=1)]
        prediction = self.fc_out(output.squeeze(0))
        #prediction = [batch size, vocab size]
        hidden = hidden.squeeze(0)
        #hidden = [batch size, hid dim]
        return prediction, hidden

In [79]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

encoder = RNNpart1(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, PAD_IDX)
decoder = RNNdecoder(HIDDEN_DIM, EMBEDDING_DIM, VOCAB_SIZE, PAD_IDX)
# print(encoder)
# print(classifer)

In [80]:
encoder.load_state_dict(torch.load('encoder.pt'))

<All keys matched successfully>

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
decoder.embedding.weight.data.copy_(pretrained_embeddings)

In [81]:
import torch.optim as optim
import itertools

optimizer = optim.SGD(decoder.parameters(), lr=1e-3)

TEXT_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

x_loss = nn.CrossEntropyLoss(ignore_index = TEXT_PAD_IDX)
z_loss = nn.MSELoss(reduction='mean')
x_wt = random.random()

encoder = encoder.to(device)

In [ ]:
# decoder.embedding.weight.requires_grad = False

In [99]:
def train_decoder(encoder, decoder , iterator, optimizer, device, x_loss, z_loss, x_wt, clip):
    
    epoch_loss = 0
    
    decoder.train()
    
    for i,batch in enumerate(iterator):
        
        
        optimizer.zero_grad()
        
        text = batch.text
        #text = [sent len, batch size]
        sent_len, batch_size = text.shape
        
        hidden_vec = encoder(text) 
        # hidden_vec = [batch_size, hid dim]
        
        trg_len = min(sent_len, 32)

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, VOCAB_SIZE).to(device)   
        pred_text = torch.zeros(trg_len, batch_size, dtype=torch.long).to(device) 
        #last hidden state of the encoder is used as the initial hidden state of the decoder

        #first input to the decoder is the <sos> tokens
        wrd = torch.LongTensor([ TEXT.vocab.stoi[TEXT.init_token] ]*batch_size)
        pred_text[0] = TEXT.vocab.stoi[TEXT.init_token]
        hidden = hidden_vec

        for t in range(1, trg_len):

            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden = decoder(hidden, wrd)
            #output = [batch size, vocab size]
            #hidden = [batch_size, hid dim]
        

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing ratio = 0.5
            teacher_force = random.random() < 0.5

            #get the highest predicted token from our predictions
            top_wrd = output.argmax(1) 
            pred_text[t] = top_wrd

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            wrd = text[t] if teacher_force else top_wrd
        
        hidden_vec1 = encoder(pred_text)
        #text = [sent len, batch size]
        #pred_text = [trg_len = min(sent_len, 32), batch size]

        outputs = outputs[1:].view(-1, VOCAB_SIZE)
        text1 = text[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
#         print(trg.shape,output.shape)
        trg_len = outputs.shape[0]
        trg = text1[:trg_len]
        
        loss = x_wt*x_loss(outputs, trg) + (1-x_wt)*z_loss(hidden_vec, hidden_vec1)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
      

        if i%50 == 0 :
            print('batch_no :',i)
        
    return epoch_loss / len(iterator)

In [103]:
def evaluate_decoder(encoder, decoder , iterator, device, x_loss, z_loss, x_wt):
    
    epoch_loss = 0
    
    decoder.eval()
    
    for i,batch in enumerate(iterator):
        
        text = batch.text
        #text = [sent len, batch size]
        sent_len, batch_size = text.shape
        
        hidden_vec = encoder(text) 
        # hidden_vec = [batch_size, hid dim]
        
        trg_len = 32

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, VOCAB_SIZE).to(device)   
        pred_text = torch.zeros(trg_len, batch_size, dtype=torch.long).to(device) 
        #last hidden state of the encoder is used as the initial hidden state of the decoder

        #first input to the decoder is the <sos> tokens
        wrd = torch.LongTensor([ TEXT.vocab.stoi[TEXT.init_token] ]*batch_size)
        pred_text[0] = TEXT.vocab.stoi[TEXT.init_token]
        hidden = hidden_vec

        for t in range(1, trg_len):

            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden = decoder(hidden, wrd)
            #output = [batch size, vocab size]
            #hidden = [batch_size, hid dim]
        

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #get the highest predicted token from our predictions
            top_wrd = output.argmax(1) 
            pred_text[t] = top_wrd

  
        hidden_vec1 = encoder(pred_text)
        #text = [sent len, batch size]
        #pred_text = [trg_len = min(sent_len, 32), batch size]

        outputs = outputs[1:].view(-1, VOCAB_SIZE)
        text1 = text[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
#         print(trg.shape,output.shape)
        trg_len = outputs.shape[0]
        trg = text1[:trg_len]
        
        loss = x_wt*x_loss(outputs, trg) + (1-x_wt)*z_loss(hidden_vec, hidden_vec1)
        
        epoch_loss += loss.item()
      

        if i%50 == 0 :
            print('batch_no :',i)
        
    return epoch_loss / len(iterator)

In [ ]:
N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss = train_decoder(encoder, decoder, train_iterator, optimizer, device, x_loss, z_loss, x_wt, CLIP)
    valid_loss = evaluate_decoder(encoder, decoder, valid_iterator, device, x_loss, z_loss, x_wt)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(decoder_model.state_dict(), 'decoder.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')

batch_no : 0


In [15]:
_tr, valid_iterator2 = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = 1,
    device = device)


In [28]:
class FGSM:
    """
    Fast Gradient Sign Method
    Ian J. Goodfellow, Jonathon Shlens, Christian Szegedy.
    Explaining and Harnessing Adversarial Examples.
    ICLR, 2015
    """
    def __init__(self, eps=0.15, clip_max=0.5, clip_min=-0.5):
#         super(FGSM, self).__init__(clip_max, clip_min)
        self.clip_max = clip_max
        self.clip_min = clip_min
        self.eps = eps
        self.fun = self.Function
    def Function (self, x):
        
        
    def generate(self, model, criterion, x, y):
        model.eval()
        nx = x.clone().detach()
        ny = y
#         nx = torch.unsqueeze(x, 0)
#         ny = torch.unsqueeze(y, 0)
        nx.requires_grad_()
        out = model(nx).squeeze(1)
#         loss = F.cross_entropy(out, ny)
        loss = criterion(out, ny)
        loss.backward()
        x_adv = nx + self.eps * torch.sign(nx.grad.data)
        x_adv.clamp_(self.clip_min, self.clip_max)
#         x_adv.squeeze_(0)
        return x_adv.detach()

In [21]:
class BIM:
    """
    Basic Iterative Method
    Alexey Kurakin, Ian J. Goodfellow, Samy Bengio.
    Adversarial Examples in the Physical World.
    arXiv, 2016
    """
    def __init__(self, eps=0.15, eps_iter=0.01, n_iter=50, clip_max=0.5, clip_min=-0.5):
        self.clip_max = clip_max
        self.clip_min = clip_min
#         super(BIM, self).__init__(clip_max, clip_min)
        self.eps = eps
        self.eps_iter = eps_iter
        self.n_iter = n_iter
#         self.device = device

    def generate(self, model, criterion, x, y):
        model.eval()
#         x.to(self.device)
        nx = x.clone().detach()
        ny = y
#         nx = torch.unsqueeze(x, 0)
#         ny = torch.unsqueeze(y, 0)
        nx.requires_grad_()
        eta = torch.zeros(nx.shape)
        if torch.cuda.is_available():
            eta = eta.cuda()
        for i in range(self.n_iter):
            out = model(nx+eta).squeeze(1)
#             loss = F.cross_entropy(out, ny)
            loss = criterion(out, ny)
            loss.backward()
            eta += self.eps_iter * torch.sign(nx.grad.data)
            eta.clamp_(-self.eps, self.eps)
            nx.grad.data.zero_()
        x_adv = nx + eta
        x_adv.clamp_(self.clip_min, self.clip_max)
#         x_adv.squeeze_(0)
        return x_adv.detach()

In [46]:
class DeepFool():
    """
    DeepFool
    Seyed-Mohsen Moosavi-Dezfooli, Alhussein Fawzi, Pascal Frossard
    DeepFool: A Simple and Accurate Method to Fool Deep Neural Networks.
    CVPR, 2016
    """
    def __init__(self, max_iter=50, clip_max=0.5, clip_min=-0.5):
#         super(DeepFool, self).__init__(clip_max, clip_min)
        self.clip_max = clip_max
        self.clip_min = clip_min
        self.max_iter = max_iter

    def generate(self, model, criterion, x, y):
        model.eval()
        nx = x.clone().detach()
        nx.requires_grad_()
        ny = y
#         eta = torch.zeros(nx.shape)
#         if torch.cuda.is_available():
#             eta = eta.cuda()
#         eta.requires_grad_()
#         out = model(nx+eta)
        for i in range(self.max_iter):
            out = model(nx).squeeze(1)
            if torch.sign(out) == torch.sign(ny):
                break
            out.backward()
            grad_np = nx.grad.data.clone()
#             print(type(nx.grad))
            nx.data += - (out * grad_np) / torch.norm(grad_np)
#             nx.clamp_(self.clip_min, self.clip_max)
#             print(type(nx.grad) )
            nx.grad.data.zero_()
        x_adv = nx 
        x_adv.requires_grad_(False)
        x_adv.clamp_(self.clip_min, self.clip_max)
#         x_adv.squeeze_(0)
        return x_adv.detach()

In [50]:

attack = DeepFool()
# attack = FGSM()
# attack = BIM()
for batch in valid_iterator2:
    z = model1(batch.text)

    z_clone = z.clone().detach()
#     print(z_clone.is_cuda)
#     break
    z_new = attack.generate(model2, criterion, z_clone, batch.label)
#     print(z)
#     print(z_new)
    y = model2(z)
    y_new = model2(z_new)
    print(y.data, y_new.data, batch.label)

    break

tensor([[-0.1221]], device='cuda:0') tensor([[0.0024]], device='cuda:0') tensor([0.], device='cuda:0')
